# Sequence Classification

In [ ]:
!pip install datasets
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.2 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer,Trainer, TrainingArguments, DistilBertForSequenceClassification, DistilBertTokenizerFast, DataCollatorWithPadding, pipeline
from datasets import Dataset, load_dataset
from evaluate import load
import numpy as np

## Load SNIPS dataset

In [ ]:
snips_file = open('snips_train.txt', 'rb')
snips_rows = snips_file.readlines()

In [ ]:
def extract_from_snip(data, start_word):
    if start_word in data:
        start_index = data.index(start_word)
        return data[start_index+1: -1]  # Son virgüle kadar al
    return []
def filter_snips(data,start_word,end_word):
  start = snip_row.find(start_word) + len(start_word)
  end = snip_row.find(end_word)
  snip = snip_row[start:end].strip()
  return snip

In [ ]:
utterances = []
tokenized_utterances = []
labels_for_tokens = []
sequence_labels = []

for snip_row in snips_rows:
    snip_row = snip_row.decode()
    snip = filter_snips(snip_row,"BOS","EOS")
    utterances.append(snip)
    tokenized_utterances.append(snip.split())
    labels_for_tokens.append(extract_from_snip(snip_row.split(' '),"EOS\tO"))
    sequence_labels.append(snip_row.split(" ")[-1].strip())

In [ ]:
print(tokenized_utterances[0])
print(labels_for_tokens[0])
print(utterances[0])
print(sequence_labels[0])

['listen', 'to', 'westbam', 'alumb', 'allergic', 'on', 'google', 'music']
['O', 'O', 'B-artist', 'O', 'B-album', 'O', 'B-service', 'I-service']
listen to westbam alumb allergic on google music
PlayMusic


In [ ]:
len(labels_for_tokens), len(tokenized_utterances), len(utterances), len(sequence_labels)

(13084, 13084, 13084, 13084)

In [ ]:
# getting all sequence labels
unique_sequence_labels = list(set(sequence_labels))
unique_sequence_labels

['BookRestaurant',
 'GetWeather',
 'AddToPlaylist',
 'SearchCreativeWork',
 'PlayMusic',
 'SearchScreeningEvent',
 'RateBook']

In [ ]:
sequence_labels = [unique_sequence_labels.index(l) for l in sequence_labels]
print((f'There are {len(unique_sequence_labels)} unique labels: {unique_sequence_labels}'))

There are 7 unique labels: ['BookRestaurant', 'GetWeather', 'AddToPlaylist', 'SearchCreativeWork', 'PlayMusic', 'SearchScreeningEvent', 'RateBook']


In [ ]:
from functools import reduce
unique_token_labels = list(set(reduce(lambda x, y: x+y, labels_for_tokens)))
labels_for_tokens = [[unique_token_labels.index(_) for _ in l] for l in labels_for_tokens]
print(f'There are {len(unique_token_labels)} unique token labels: {unique_token_labels}')

There are 72 unique token labels: ['B-cuisine', 'B-party_size_description', 'B-track', 'B-service', 'I-facility', 'B-object_part_of_series_type', 'I-object_type', 'B-city', 'I-movie_type', 'B-object_name', 'B-condition_temperature', 'I-object_location_type', 'B-condition_description', 'I-restaurant_name', 'I-location_name', 'B-rating_unit', 'B-object_select', 'B-movie_name', 'B-album', 'B-genre', 'I-timeRange', 'B-rating_value', 'B-playlist', 'I-object_select', 'I-restaurant_type', 'B-facility', 'I-geographic_poi', 'B-music_item', 'I-track', 'I-current_location', 'I-service', 'I-playlist', 'I-playlist_owner', 'B-current_location', 'B-spatial_relation', 'B-restaurant_type', 'I-object_part_of_series_type', 'B-object_location_type', 'B-sort', 'I-sort', 'B-playlist_owner', 'B-best_rating', 'I-poi', 'B-timeRange', 'B-geographic_poi', 'I-genre', 'I-object_name', 'I-movie_name', 'I-album', 'O', 'B-entity_name', 'B-location_name', 'B-served_dish', 'B-poi', 'B-country', 'I-entity_name', 'B-stat

In [ ]:
print(tokenized_utterances[0])
print(labels_for_tokens[0])
print([unique_token_labels[l] for l in labels_for_tokens[0]])
print(utterances[0])
print(sequence_labels[0])
print(unique_sequence_labels[sequence_labels[0]])

['listen', 'to', 'westbam', 'alumb', 'allergic', 'on', 'google', 'music']
[49, 49, 63, 49, 18, 49, 3, 30]
['O', 'O', 'B-artist', 'O', 'B-album', 'O', 'B-service', 'I-service']
listen to westbam alumb allergic on google music
4
PlayMusic


## Create dataset dictionary

In [ ]:
snips_dataset = Dataset.from_dict( # holding data for both sequence and token classification
    dict(
        utterance=utterances,
        label=sequence_labels,
        tokens=tokenized_utterances,
        token_labels = labels_for_tokens
    )
)

In [ ]:
# Train and test dataset separation
snips_dataset = snips_dataset.train_test_split(test_size=0.2)

In [ ]:
snips_dataset['train'][0]

{'utterance': 'how much rain in montana 12 months from now',
 'label': 1,
 'tokens': ['how',
  'much',
  'rain',
  'in',
  'montana',
  '12',
  'months',
  'from',
  'now'],
 'token_labels': [49, 49, 12, 49, 56, 43, 20, 20, 20]}

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
# simple function to batch tokenize utterances with truncation
def preprocess_function(examples):
  return tokenizer(examples["utterance"], truncation=True)

In [ ]:
seq_clf_tokenized_snips = snips_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/10467 [00:00<?, ? examples/s]

Map:   0%|          | 0/2617 [00:00<?, ? examples/s]

In [ ]:
# DataCollatorWithPadding creates batch of data. It also dynamically pads text to the lenght of the
# longest element in the batch, making them all the same lenght.
# It's possible to pad your text in the tokneizer function with padding=True, dynamic padding is more efficient.

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Initialize model

In [ ]:
sequence_clf_model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased', num_labels=len(unique_sequence_labels),
)

# set an index -> label dictionary
sequence_clf_model.config.id2label = {i:l for i,l in enumerate(unique_sequence_labels)}

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
metric = load('accuracy')

def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions  = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

## Training model



In [ ]:
epochs = 2
training_args = TrainingArguments(
    output_dir="./snips_clf_results",
    num_train_epochs=epochs,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    load_best_model_at_end=True,

    # some deep learning parameters that the Trainer is able to take in
    warmup_steps=len(seq_clf_tokenized_snips["train"]) // 5,  # number of warmup steps for learning rate scheduler
    weight_decay=0.05,

    logging_steps=1,
    log_level='info',
    eval_strategy='epoch',
    save_strategy='epoch',
    report_to='tensorboard',
)

# Define the trainer:

trainer = Trainer(
    model=sequence_clf_model,
    args=training_args,
    train_dataset = seq_clf_tokenized_snips["train"],
    eval_dataset = seq_clf_tokenized_snips["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

PyTorch: setting up devices
<ipython-input-26-332aa61dbe53>:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance, token_labels, tokens. If utterance, token_labels, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 2617
  Batch size = 64


{'eval_loss': 1.8384268283843994,
 'eval_model_preparation_time': 0.005,
 'eval_accuracy': 0.38249904470768054,
 'eval_runtime': 102.307,
 'eval_samples_per_second': 25.58,
 'eval_steps_per_second': 0.401}

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance, token_labels, tokens. If utterance, token_labels, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 10,467
  Num Epochs = 2
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 164
  Number of trainable parameters = 66,958,855


Epoch,Training Loss,Validation Loss,Accuracy
1,1.731700,1.700329,0.627436
2,1.158800,1.081583,0.918609


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance, token_labels, tokens. If utterance, token_labels, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 2617
  Batch size = 128
Saving model checkpoint to ./snips_clf_results/checkpoint-82
Configuration saved in ./snips_clf_results/checkpoint-82/config.json
Model weights saved in ./snips_clf_results/checkpoint-82/model.safetensors
tokenizer config file saved in ./snips_clf_results/checkpoint-82/tokenizer_config.json
Special tokens file saved in ./snips_clf_results/checkpoint-82/special_tokens_map.json
Saving model checkpoint to ./snips_clf_results/checkpoint-164
Configuration saved in ./snips_clf_results/checkpoint-164/config.json
Model weights saved in ./snips_clf_results/checkpoint-164/model.safetensors
tokenizer config

TrainOutput(global_step=164, training_loss=1.6497861263228626, metrics={'train_runtime': 2939.5888, 'train_samples_per_second': 7.121, 'train_steps_per_second': 0.056, 'total_flos': 137558183978082.0, 'train_loss': 1.6497861263228626, 'epoch': 2.0})

## Testing model

In [ ]:
pipe = pipeline("text-classification", model=sequence_clf_model, tokenizer=tokenizer)
pipe('Add Two Coins by Dispatch to my road trip card')

Device set to use cpu


[{'label': 'BookRestaurant', 'score': 0.19140559434890747}]

In [ ]:
pipe = pipeline("text-classification", model=sequence_clf_model, tokenizer=tokenizer)
pipe('Add the Shivers to my playlist')

Device set to use cpu


[{'label': 'AddToPlaylist', 'score': 0.5447279810905457}]

## Saving model

In [ ]:
trainer.save_model()

Saving model checkpoint to ./snips_clf_results
Configuration saved in ./snips_clf_results/config.json
Model weights saved in ./snips_clf_results/model.safetensors
tokenizer config file saved in ./snips_clf_results/tokenizer_config.json
Special tokens file saved in ./snips_clf_results/special_tokens_map.json


# Token Classification

In [ ]:
from transformers import DataCollatorForTokenClassification, DistilBertForTokenClassification

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/tokenizer_config.json
loading file chat_template.jinja from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ]

In [ ]:
#using same dataset for token classification
snips_dataset['train'][0]

{'utterance': 'how much rain in montana 12 months from now',
 'label': 1,
 'tokens': ['how',
  'much',
  'rain',
  'in',
  'montana',
  '12',
  'months',
  'from',
  'now'],
 'token_labels': [49, 49, 12, 49, 56, 43, 20, 20, 20]}

In [ ]:
# The given "token_labels" may not match up with the BERT wordpiece tokenization so
# this function will map them to the tokenization that BERT uses
# -100 is a reserved for labels where we do not want to calculate losses so BERT dosen't waste time
# trying to predict tokens like CLS or SEP

def tokenize_and_align_labels(examples):
  tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)

  labels = []
  for i, label in enumerate(examples[f'token_labels']):
    word_ids = tokenized_inputs.word_ids(batch_index=i) # Map tokens to their respective word.
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:
      if word_idx is None: # Seet any special tokens / punctution to -100
        label_ids.append(-100)
      elif word_idx != previous_word_idx: # Only label the first token of a given word.
        label_ids.append(label[word_idx])
      else:
        label_ids.append(-100)
      previous_word_idx = word_idx
    labels.append(label_ids)

  tokenized_inputs['labels'] = labels
  return tokenized_inputs

In [ ]:
tok_clf_tokenized_snips = snips_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/10467 [00:00<?, ? examples/s]

Map:   0%|          | 0/2617 [00:00<?, ? examples/s]

In [ ]:
tok_clf_tokenized_snips["train"][0]

{'utterance': 'how much rain in montana 12 months from now',
 'label': 1,
 'tokens': ['how',
  'much',
  'rain',
  'in',
  'montana',
  '12',
  'months',
  'from',
  'now'],
 'token_labels': [49, 49, 12, 49, 56, 43, 20, 20, 20],
 'input_ids': [101, 2129, 2172, 4542, 1999, 8124, 2260, 2706, 2013, 2085, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 49, 49, 12, 49, 56, 43, 20, 20, 20, -100]}

In [ ]:
# We don't need ['tokens', token_labels','utterance', 'labels'] columns
tok_clf_tokenized_snips['train'] = tok_clf_tokenized_snips['train'].remove_columns(['tokens','token_labels', 'utterance', 'label'])
tok_clf_tokenized_snips['test'] = tok_clf_tokenized_snips['test'].remove_columns(['tokens','token_labels', 'utterance', 'label'])

In [ ]:
tok_clf_tokenized_snips

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10467
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2617
    })
})

In [ ]:
tok_data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

## Loading Bert pretrained model

In [ ]:
tok_clf_model = DistilBertForTokenClassification.from_pretrained(
    'distilbert-base-uncased', num_labels=len(unique_token_labels)
)

#Set our label dictionary
tok_clf_model.config.id2label = {i: l for i, l in enumerate(unique_token_labels)}

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25": "LABEL_25",
    "26": "LABEL_26",
    "27": "LABEL_27",
    "28": "LABEL_28

## Training model

In [ ]:
epochs = 2
training_args = TrainingArguments(
    output_dir="./snips_tok_clf_results",
    num_train_epochs=epochs,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    load_best_model_at_end=True,

    logging_steps=10,
    log_level='info',
    eval_strategy='epoch',
    save_strategy='epoch',
    report_to='tensorboard',
)

# Define the trainer:

trainer = Trainer(
    model=tok_clf_model,
    args=training_args,
    train_dataset = tok_clf_tokenized_snips["train"],
    eval_dataset = tok_clf_tokenized_snips["test"],
    data_collator=tok_data_collator,
)

PyTorch: setting up devices


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 10,467
  Num Epochs = 2
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 164
  Number of trainable parameters = 66,418,248


Epoch,Training Loss,Validation Loss
1,0.725700,0.593627
2,0.403200,0.355072



***** Running Evaluation *****
  Num examples = 2617
  Batch size = 128
Saving model checkpoint to ./snips_tok_clf_results/checkpoint-82
Configuration saved in ./snips_tok_clf_results/checkpoint-82/config.json
Model weights saved in ./snips_tok_clf_results/checkpoint-82/model.safetensors
Saving model checkpoint to ./snips_tok_clf_results/checkpoint-164
Configuration saved in ./snips_tok_clf_results/checkpoint-164/config.json
Model weights saved in ./snips_tok_clf_results/checkpoint-164/model.safetensors

***** Running Evaluation *****
  Num examples = 2617
  Batch size = 128
Saving model checkpoint to ./snips_tok_clf_results/checkpoint-164
Configuration saved in ./snips_tok_clf_results/checkpoint-164/config.json
Model weights saved in ./snips_tok_clf_results/checkpoint-164/model.safetensors


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./snips_tok_clf_results/checkpoint-164 (score: 0.35507234930992126).


TrainOutput(global_step=164, training_loss=1.021860132130181, metrics={'train_runtime': 2906.6444, 'train_samples_per_second': 7.202, 'train_steps_per_second': 0.056, 'total_flos': 135833769825264.0, 'train_loss': 1.021860132130181, 'epoch': 2.0})

In [46]:
trainer.evaluate()


***** Running Evaluation *****
  Num examples = 2617
  Batch size = 128


{'eval_loss': 0.35507234930992126,
 'eval_runtime': 155.7057,
 'eval_samples_per_second': 16.807,
 'eval_steps_per_second': 0.135,
 'epoch': 2.0}

## Testing model

In [47]:
pipe = pipeline("token-classification", tok_clf_model, tokenizer=tokenizer)
pipe('Add Two Coins by Dispatch to my road trip card')

Device set to use cpu
Disabling tokenizer parallelism, we're using DataLoader multithreading already


[{'entity': 'B-rating_value',
  'score': 0.45280528,
  'index': 2,
  'word': 'two',
  'start': 4,
  'end': 7},
 {'entity': 'B-music_item',
  'score': 0.20003434,
  'index': 3,
  'word': 'coins',
  'start': 8,
  'end': 13},
 {'entity': 'I-entity_name',
  'score': 0.3953059,
  'index': 5,
  'word': 'dispatch',
  'start': 17,
  'end': 25},
 {'entity': 'B-playlist_owner',
  'score': 0.89096725,
  'index': 7,
  'word': 'my',
  'start': 29,
  'end': 31},
 {'entity': 'B-playlist',
  'score': 0.877202,
  'index': 8,
  'word': 'road',
  'start': 32,
  'end': 36},
 {'entity': 'I-playlist',
  'score': 0.89761853,
  'index': 9,
  'word': 'trip',
  'start': 37,
  'end': 41},
 {'entity': 'I-playlist',
  'score': 0.6959232,
  'index': 10,
  'word': 'card',
  'start': 42,
  'end': 46}]

In [48]:
pipe = pipeline("token-classification", tok_clf_model, tokenizer=tokenizer)
pipe('Rate the principles of the data science 5 out of 5')

Device set to use cpu


[{'entity': 'B-object_name',
  'score': 0.86016625,
  'index': 2,
  'word': 'the',
  'start': 5,
  'end': 8},
 {'entity': 'I-object_name',
  'score': 0.9279909,
  'index': 3,
  'word': 'principles',
  'start': 9,
  'end': 19},
 {'entity': 'I-object_name',
  'score': 0.95985425,
  'index': 4,
  'word': 'of',
  'start': 20,
  'end': 22},
 {'entity': 'I-object_name',
  'score': 0.9576993,
  'index': 5,
  'word': 'the',
  'start': 23,
  'end': 26},
 {'entity': 'I-object_name',
  'score': 0.96416026,
  'index': 6,
  'word': 'data',
  'start': 27,
  'end': 31},
 {'entity': 'I-object_name',
  'score': 0.96163154,
  'index': 7,
  'word': 'science',
  'start': 32,
  'end': 39},
 {'entity': 'B-rating_value',
  'score': 0.9540878,
  'index': 8,
  'word': '5',
  'start': 40,
  'end': 41},
 {'entity': 'B-best_rating',
  'score': 0.922161,
  'index': 11,
  'word': '5',
  'start': 49,
  'end': 50}]